In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import os

In [2]:
base = "/Users/nickichen/Desktop/urap data"

#loading datasets
cleaned_names = pd.read_csv(f"{base}/bunmd_clean_names_v1.csv.gz", compression="gzip")
geography = pd.read_csv(f"{base}/bunmd_geography_supplement_v1.csv")
sibs_exact = pd.read_csv(f"{base}/bunmd_siblings_v2/bunmd_sibs_exact_match_v2.csv")
sibs_flexible = pd.read_csv(f"{base}/bunmd_siblings_v2/bunmd_sibs_flexible_match_v2.csv")
database = pd.read_csv(f"{base}/bunmd_v2.csv")

/var/folders/ng/ffs689f95vx8r67ys3wq6zzr0000gn/T/ipykernel_3564/2108617447.py:8: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  database = pd.read_csv(f"{base}/bunmd_v2.csv")


In [3]:
import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('io.excel.xlsx.writer', 'xlsxwriter')

In [4]:
import pandas as pd
import numpy as np
import os
import resource

# Silence pandas warnings
pd.options.mode.chained_assignment = None
pd.set_option('io.excel.xlsx.writer', 'xlsxwriter')

# Extend memory limit (18 GB on your Mac)
soft, hard = resource.getrlimit(resource.RLIMIT_AS)
try:
    resource.setrlimit(resource.RLIMIT_AS, (18 * 1024**3, hard))
except Exception as e:
    print("⚠️ Could not set memory limit (normal on macOS sandboxed Jupyter):", e)

# Install Parquet engine if needed
try:
    import pyarrow
except ImportError:
    !pip install pyarrow


⚠️ Could not set memory limit (normal on macOS sandboxed Jupyter): current limit exceeds maximum limit


In [6]:
import pandas as pd

base = "/Users/nickichen/Desktop/urap data"

# Load once and sort permanently
print("🔄 Sorting the full dataset by birth year, last name, and first name...")
df = pd.read_csv(f"{base}/bunmd_v2.csv", low_memory=False)
df.sort_values(["byear", "lname", "fname"], inplace=True)
df.to_parquet(f"{base}/bunmd_sorted.parquet", index=False)
print("✅ Saved sorted version: bunmd_sorted.parquet")


🔄 Sorting the full dataset by birth year, last name, and first name...


✅ Saved sorted version: bunmd_sorted.parquet


In [29]:
import pandas as pd
import numpy as np
import os

def find_yearbook_person_fast(
    first_names,
    middle_names,
    last_names,
    gender="unknown",
    yearbook_year=None,
    cleaned_names=None,
    geography=None,
    database_path=None
):
    """
    only loads the relevant birth-year window from the pre-sorted Parquet database.
    """

    # Normalize inputs
    if isinstance(first_names, str):
        first_names = [first_names]
    if isinstance(middle_names, str):
        middle_names = [middle_names]
    if isinstance(last_names, str):
        last_names = [last_names]

    first_names = [f.lower().strip() for f in first_names]
    last_names = [l.lower().strip() for l in last_names]

    expanded_mnames = set()
    for m in middle_names:
        m = m.lower().strip()
        if not m:
            expanded_mnames.add("")
        else:
            expanded_mnames.update(["", m[0], m])
    middle_names = list(expanded_mnames)

    print(f"\n--- Searching for {first_names} {middle_names} {last_names} | Gender: {gender} ---")

    # Expected birth-year range
    if yearbook_year:
        birth_low = yearbook_year - 19
        birth_high = yearbook_year - 13
        print(f"Expected birth years: {birth_low}–{birth_high}")
    else:
        birth_low = birth_high = None
        print("No yearbook year given → no birth-year filter.")

    # Load only relevant rows from the pre-sorted Parquet file
    parquet_path = database_path.replace(".csv", "_sorted.parquet")
    if not os.path.exists(parquet_path):
        raise FileNotFoundError(f"Missing sorted Parquet file: {parquet_path}")

    filters = []
    if birth_low and birth_high:
        filters = [("byear", ">=", birth_low), ("byear", "<=", birth_high)]

    print("📂 Loading relevant subset from Parquet...")
    database = pd.read_parquet(parquet_path, filters=filters, engine="pyarrow")

    # Filter by names (case-insensitive)
    mask = (
        database["fname"].str.lower().isin(first_names)
        & database["lname"].str.lower().isin(last_names)
        & database["mname"].str.lower().isin(middle_names)
    )
    db = database.loc[mask].copy()
    print(f"✅ Found {len(db)} matching name records.")

    # Add how the person matched (birth vs death)
    if not db.empty and geography is not None:
        db = db.merge(
            geography[["ssn", "birth_fips", "death_state", "death_zip"]],
            on="ssn", how="left"
        )
        db["match_type"] = np.where(
        db["birth_fips"].astype(str).str.startswith("37", na=False),
        "birth_state",
        np.where(
            db["death_state"].str.lower() == "nc",
            "death_state",
            "other"
        )
    )


    # Filter by birth-year range (redundant but safe)
    if not db.empty and birth_low:
        db = db[db["byear"].between(birth_low, birth_high, inclusive="both")]

    return db

      


In [12]:
import sys
!{sys.executable} -m pip install pyarrow --upgrade

import pyarrow
print("✅ pyarrow loaded successfully!")


Python(87009) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: pip3.12 install --upgrade pip
✅ pyarrow loaded successfully!


In [47]:
# === Run batch search for The Chief 1939 yearbook ===

import pandas as pd

yearbook_year = 1939
base = "/Users/nickichen/Desktop/urap data"

all_results = []

# All people and corresponding genders
people = [
    # Page 12
    ("edith", "", "allison"),
    ("iola", "bobette", "anders"),
    ("julia", "jeanette", "anders"),
    ("charlie", "g", "banks"),
    ("albert", "monroe", "cantrell"),
    ("ada", "louise", "dalton"),

    # Pages 13–15
    ("janie", "lou", "dalton"),
    ("troy", "a", "drake"),
    ("frank", "r", "gash"),
    ("george", "w", "gibbs"),
    ("helen", "virginia", "holtzclaw"),
    ("margaret", "arlola", "jenkins"),
    ("james", "terry", "mann"),
    ("nell", "", "merrill"),
    ("lillian", "", "mintz"),
    ("mildred", "catherine", "mccarson"),
    ("wayne", "", "nicholson"),
    ("virginia", "", "noonchester"),
    ("cleatus", "", "norton"),
    ("sue", "eileen", "orr"),
    ("ellis", "", "revis"),
    ("fannie", "", "summey"),
    ("jessie", "", "stamey"),
    ("agnes", "frances", "whiteside")
]

genders = [
    # Page 12
    "female", "female", "female", "male", "male", "female",
    # Pages 13–15
    "female", "male", "male", "male", "female", "female",
    "male", "female", "female", "female", "male", "female",
    "male", "female", "male", "female", "female", "female"
]

# === Run the searches ===
for (first, middle, last), gender in zip(people, genders):
    print(f"\n--- Searching for {first.title()} {middle.title()} {last.title()} ({gender}) ---")

    df = find_yearbook_person_fast(
        first_names=first,
        middle_names=middle,
        last_names=last,
        gender=gender,
        yearbook_year=yearbook_year,
        cleaned_names=cleaned_names,
        geography=geography,
        database_path=f"{base}/bunmd.csv"

    )

    if not df.empty:
        df["person_name"] = f"{first.title()} {middle.title()} {last.title()}".replace("  ", " ")
        df["gender"] = gender
        df["yearbook"] = "The Chief 1939"
        all_results.append(df)

# === Combine everything ===
if all_results:
    combined_results = pd.concat(all_results, ignore_index=True)
    print(f"\n✅ Combined {len(all_results)} matched individuals from The Chief 1939.")
    display(combined_results.head(100))
else:
    print("⚠️ No matches found for any listed names.")


--- Searching for Edith  Allison (female) ---

--- Searching for ['edith'] [''] ['allison'] | Gender: female ---
Expected birth years: 1920–1926
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Iola Bobette Anders (female) ---

--- Searching for ['iola'] ['', 'bobette', 'b'] ['anders'] | Gender: female ---
Expected birth years: 1920–1926
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Julia Jeanette Anders (female) ---

--- Searching for ['julia'] ['', 'jeanette', 'j'] ['anders'] | Gender: female ---
Expected birth years: 1920–1926
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Charlie G Banks (male) ---

--- Searching for ['charlie'] ['', 'g'] ['banks'] | Gender: male ---
Expected birth years: 1920–1926
📂 Loading relevant subset from Parquet...
✅ Found 2 matching name records.

--- Searching for Albert Monroe Cantrell (male) ---

--- Searching for 

,ssn,zip_residence,lname,mname,fname,byear,dyear,socstate,bmonth,dmonth,...,ccweight,bpl_string,socstate_string,birth_fips,death_state,death_zip,match_type,person_name,gender,yearbook
0,569307270,287420245,BANKS,G,CHARLIE,1922,2002,600.0,3.0,1,...,1.065031,North Carolina,California,37001.0,nc,28742,birth_state,Charlie G Banks,male,The Chief 1939
1,427125168,441124404,BANKS,G,CHARLIE,1924,1991,2800.0,9.0,6,...,1.149759,Mississippi,Mississippi,28163.0,oh,44112,other,Charlie G Banks,male,The Chief 1939
2,402247221,438321145,CANTRELL,M,ALBERT,1921,1998,2100.0,8.0,8,...,1.099517,Kentucky,Kentucky,21195.0,oh,43832,other,Albert Monroe Cantrell,male,The Chief 1939
3,90166074,105664630,GIBBS,W,GEORGE,1920,1999,3600.0,3.0,5,...,1.103359,New York,New York,36061.0,ny,10566,other,George W Gibbs,male,The Chief 1939
4,231036052,22736,GIBBS,W,GEORGE,1921,1986,5100.0,9.0,12,...,NaN,Virginia,Virginia,NaN,va,22736,other,George W Gibbs,male,The Chief 1939
5,334122300,None,GIBBS,W,GEORGE,1924,2002,1700.0,3.0,1,...,1.057891,Michigan,Illinois,26163.0,NaN,NaN,other,George W Gibbs,male,The Chief 1939
6,335249104,857428739,JENKINS,A,MARGARET,1922,1998,1700.0,8.0,2,...,1.088224,Illinois,Illinois,17183.0,az,85742,other,Margaret Arlola Jenkins,female,The Chief 1939
7,144142673,088073026,JENKINS,A,MARGARET,1922,1999,3400.0,3.0,5,...,1.089540,Pennsylvania,New Jersey,42095.0,nj,08807,other,Margaret Arlola Jenkins,female,The Chief 1939
8,176265512,080814914,JENKINS,A,MARGARET,1923,1996,4200.0,11.0,7,...,1.085057,Pennsylvania,Pennsylvania,42091.0,nj,08081,other,Margaret Arlola Jenkins,female,The Chief 1939
9,265825171,344713333,JENKINS,A,MARGARET,1923,2007,1200.0,6.0,3,...,NaN,Florida,Florida,12057.0,fl,34471,other,Margaret Arlola Jenkins,female,The Chief 1939


In [31]:
# === Run batch search for Shelby 1940 yearbook ===

import pandas as pd

yearbook_year = 1940
base = "/Users/nickichen/Desktop/urap data"

all_results = []

# === All people and corresponding genders (Shelby 1940) ===
people = [
    # shelby1940 (predominantly Black)
    ("mildred", "t", "bankhead"),
    ("essie", "c", "bass"),
    ("geneva", "elmira", "borders"),
    ("ada", "bell", "butts"),
    ("willie", "james", "butts"),

    # Pages 6–8
    ("warren", "gibson", "cadeness"),
    ("allie", "christine", "cannady"),
    ("faye", "", "cornwell"),
    ("beulah", "estelle", "earl"),
    ("boyd", "josiah", "gatheright"),

    ("charles", "evans", "hooper"),
    ("betsy", "glendylon", "hoote"),
    ("priscilla", "jane", "johnson"),
    ("nedra", "elizabeth", "keaton"),
    ("theopia", "", "littlejohn"),

    ("hester", "leonia", "logan"),
    ("mary", "elizabeth", "london"),
    ("herbert", "marshall", "roper"),
    ("lula", "mae", "teamer"),
    ("joseph", "h", "teamer"),
    ("bessie", "rhea", "turner")
]

genders = [
    # shelby1940
    "female", "female", "female", "female", "male",

    # Pages 6–8
    "male", "female", "female", "female", "male",
    "male", "female", "female", "female", "female",
    "female", "female", "male", "female", "male", "female"
]

# === Run the searches ===
for (first, middle, last), gender in zip(people, genders):
    print(f"\n--- Searching for {first.title()} {middle.title()} {last.title()} ({gender}) ---")

    df = find_yearbook_person_fast(
        first_names=first,
        middle_names=middle,
        last_names=last,
        gender=gender,
        yearbook_year=yearbook_year,
        cleaned_names=cleaned_names,
        geography=geography,
        database_path=f"{base}/bunmd_sorted.parquet"
    )

    if not df.empty:
        df["person_name"] = f"{first.title()} {middle.title()} {last.title()}".replace("  ", " ")
        df["gender"] = gender
        df["yearbook"] = "Shelby 1940"
        all_results.append(df)

# === Combine everything ===
if all_results:
    combined_results = pd.concat(all_results, ignore_index=True)
    print(f"\n✅ Combined {len(all_results)} matched individuals from Shelby 1940.")
    display(combined_results.head(100))
else:
    print("⚠️ No matches found for any listed names.")



--- Searching for Mildred T Bankhead (female) ---

--- Searching for ['mildred'] ['', 't'] ['bankhead'] | Gender: female ---
Expected birth years: 1921–1927
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Essie C Bass (female) ---

--- Searching for ['essie'] ['', 'c'] ['bass'] | Gender: female ---
Expected birth years: 1921–1927
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Geneva Elmira Borders (female) ---

--- Searching for ['geneva'] ['', 'e', 'elmira'] ['borders'] | Gender: female ---
Expected birth years: 1921–1927
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Ada Bell Butts (female) ---

--- Searching for ['ada'] ['', 'bell', 'b'] ['butts'] | Gender: female ---
Expected birth years: 1921–1927
📂 Loading relevant subset from Parquet...
✅ Found 1 matching name records.

--- Searching for Willie James Butts (male) ---

--- Searching for ['wi

,ssn,zip_residence,lname,mname,fname,byear,dyear,socstate,bmonth,dmonth,...,ccweight,bpl_string,socstate_string,birth_fips,death_state,death_zip,match_type,person_name,gender,yearbook
0,253723195,303317324,BUTTS,B,ADA,1925,2005,1300.0,3.0,8,...,1.134173,Georgia,Georgia,13197.0,ga,30331,other,Ada Bell Butts,female,Shelby 1940
1,240224211,None,BUTTS,J,WILLIE,1921,1991,3700.0,4.0,7,...,1.170743,Georgia,North Carolina,13021.0,NaN,NaN,other,Willie James Butts,male,Shelby 1940
2,256242804,349528121,BUTTS,J,WILLIE,1922,1997,1300.0,9.0,10,...,1.100285,Georgia,Georgia,13175.0,fl,34952,other,Willie James Butts,male,Shelby 1940
3,404200109,257014725,HOOPER,E,CHARLES,1922,2003,2100.0,2.0,3,...,1.065754,Kentucky,Kentucky,21133.0,wv,25701,other,Charles Evans Hooper,male,Shelby 1940
4,400266331,None,HOOPER,E,CHARLES,1923,2006,2100.0,12.0,12,...,NaN,Kentucky,Kentucky,21199.0,NaN,NaN,other,Charles Evans Hooper,male,Shelby 1940
5,557266050,925445915,JOHNSON,J,PRISCILLA,1921,1994,600.0,2.0,9,...,1.114277,Illinois,California,17031.0,ca,92544,other,Priscilla Jane Johnson,female,Shelby 1940
6,364283455,480592303,LONDON,E,MARY,1923,2001,2600.0,3.0,10,...,1.075618,Michigan,Michigan,26125.0,mi,48059,other,Mary Elizabeth London,female,Shelby 1940


In [76]:
# new function to check whole yearbooks

def find_yearbook_all_classes(yearbook_year, class_groups, database_path, save_dir):
    all_results = []

    for class_name, people, genders in class_groups:
        print(f"\n===== 📘 Processing {class_name.title()} Class =====")

        class_results = []

        for (first, middle, last), gender in zip(people, genders):
            print(f"\n--- Searching for {first.title()} {middle.title()} {last.title()} ({gender}) ---")
            df = find_yearbook_person_fast(
                first_names=first,
                middle_names=middle,
                last_names=last,
                gender=gender,
                yearbook_year=yearbook_year,
                cleaned_names=None,
                geography=None,
                database_path=database_path
            )

            if not df.empty:
                df["person_name"] = f"{first.title()} {middle.title()} {last.title()}".replace("  ", " ")
                df["gender_guess"] = gender
                df["class_level"] = class_name.title()
                df["yearbook"] = f"{yearbook_name} {yearbook_year}"
                class_results.append(df)

        if class_results:
            combined = pd.concat(class_results, ignore_index=True)
            print(f"✅ Found {len(combined)} matches for {class_name.title()}.")
            all_results.append(combined)
            os.makedirs(save_dir, exist_ok=True)
            combined.to_csv(f"{save_dir}/{class_name}_matches.csv", index=False)
        else:
            print(f"⚠️ No matches found for {class_name.title()} class.")

    combined_all, summary = None, None
    if all_results:
        combined_all = pd.concat(all_results, ignore_index=True)

        # safer display subset — only show columns that exist
        display_cols = [c for c in ["person_name", "gender_guess", "class_level", "byear", "death_state", "match_type"] if c in combined_all.columns]
        display(combined_all[display_cols].head(20))

        print(f"\n✅ Combined {len(combined_all)} total matches across all classes!")

        summary = (
            combined_all.groupby("class_level")
            .agg(total_matches=("ssn", "count"))
            .reset_index()
        )

        print("\n📊 Summary by class:")
        display(summary)

        os.makedirs(save_dir, exist_ok=True)
        combined_all.to_csv(f"{save_dir}/all_yearbooks_combined.csv", index=False)
        summary.to_csv(f"{save_dir}/call_yearbooks_summary.csv", index=False)
    else:
        print("\n⚠️ No matches found in any class.")

    return combined_all, summary


In [ ]:
#chiefs
yearbook_name = "Chiefs"
seniors = [
    ("edith", "", "allison"), ("iola", "bobette", "anders"), ("julia", "jeanette", "anders"),
    ("charlie", "g", "banks"), ("albert", "monroe", "cantrell"), ("ada", "louise", "dalton"),
    ("janie", "lou", "dalton"), ("troy", "a", "drake"), ("frank", "r", "gash"),
    ("george", "w", "gibbs"), ("helen", "virginia", "holtzclaw"), ("margaret", "arlola", "jenkins"),
    ("james", "terry", "mann"), ("nell", "", "merrill"), ("lillian", "", "mintz"),
    ("mildred", "catherine", "mccarson"), ("wayne", "", "nicholson"), ("virginia", "", "noonchester"),
    ("cleatus", "", "norton"), ("sue", "eileen", "orr"), ("ellis", "", "revis"),
    ("fannie", "", "summey"), ("jessie", "", "stamey"), ("agnes", "frances", "whiteside")
]
genders = [
    "female","female","female","male","male","female",
    "female","male","male","male","female","female",
    "male","female","female","female","male","female",
    "male","female","male","female","female","female"
]

juniors = [
    ("o", "a", "anders"), ("c", "w", "adcock"), ("franklin", "", "allison"), ("broaddeth", "", "blackwell"),
    ("claudine", "", "cairnes"), ("nellie", "", "cairnes"), ("charlotte", "", "clayton"), ("salome", "", "clayton"),
    ("jackie", "", "corn"), ("vernon", "", "conner"), ("opal", "", "dalton"), ("christine", "", "davis"),
    ("ruth", "", "gonce"), ("eula", "", "gray"), ("willie", "", "gash"), ("blanche", "", "hollingsworth"),
    ("cameron", "", "houk"), ("helen", "", "mckinna"), ("raymond", "", "mccarson"), ("alma", "", "merrill"),
    ("carolyn", "", "orr"), ("thelma", "", "snyder"), ("jesse", "lee", "simpson"), ("james", "", "smith"),
    ("clarence", "", "williams"), ("helen", "", "wilson"), ("lillian", "", "wilson"), ("lucille", "", "wilson")
]
junior_genders = [
    "male","male","male","male","female","female","female","female","female","male","female","female",
    "female","female","male","female","male","female","male","female","female","female","male","male",
    "male","female","female","female"
]

sophomores = [
    ("imogene", "", "allen"), ("wanda", "", "anders"), ("maudina", "", "bell"), ("hamilton", "", "boyd"),
    ("cecil", "", "cairnes"), ("irene", "", "corn"), ("eddie", "lee", "corn"), ("opal", "", "dalton"),
    ("coy", "", "dalton"), ("palmer", "", "drake"), ("ellis", "", "garren"), ("plato", "", "hall"),
    ("maxine", "", "hollingsworth"), ("hamilton", "", "hollingsworth"), ("ralph", "", "jones"),
    ("j", "b", "laughter"), ("wade", "", "laughter"), ("charles", "", "ledbetter"), ("freda", "", "marona"),
    ("joyce", "", "mckinna"), ("lottie", "", "mintz"), ("dorothy", "", "simpson"), ("ruth", "", "stamey"),
    ("glen", "", "simpson"), ("lillie", "mae", "smith")
]
sophomore_genders = [
    "female","female","female","male","male","female","female","female","male","male","male","male",
    "female","male","male","male","male","male","female","female","female","female","female","male","female"
]

freshmen = [
    ("louise", "", "adoeck"), ("lucille", "", "allison"), ("stella", "", "allison"), ("vera", "", "allison"),
    ("rosella", "", "ball"), ("perry", "", "bell"), ("henrietta", "", "briggs"), ("harvey", "", "cantrell"),
    ("rosa", "mae", "capell"), ("jack", "", "dalton"), ("julia", "", "davis"), ("henry", "", "garren"),
    ("rolland", "", "gibson"), ("de", "foix", "greer"), ("mildred", "", "holden"), ("j", "p", "huggins"),
    ("billy", "", "mckinna"), ("george", "", "moffitt"), ("paul", "", "moffitt"), ("mary", "alice", "merrill"),
    ("marjorie", "", "merrill"), ("cecil", "", "morgan"), ("ulysses", "", "nelson"), ("richard", "", "norris"),
    ("swannie", "", "price"), ("richard", "", "sentell"), ("marvin", "", "simpson"), ("joe", "", "smathers"),
    ("jessie", "", "snyder"), ("mildred", "", "whitesides"), ("nancy", "lee", "wright")
]
freshmen_genders = [
    "female","female","female","female","female","male","female","male","female","male","female","male",
    "male","male","female","male","male","male","male","female","female","male","male","male",
    "female","male","male","male","female","female","female"
]

# === RUN ===
base = "/Users/nickichen/Desktop/urap data"
yearbook_year = 1939

class_groups = [
    ("senior", seniors, genders),
    ("junior", juniors, junior_genders),
    ("sophomore", sophomores, sophomore_genders),
    ("freshman", freshmen, freshmen_genders)
]

combined_all, summary = find_yearbook_all_classes(
    yearbook_year=yearbook_year,
    class_groups=class_groups,
    database_path=f"{base}/bunmd_sorted.parquet",
    save_dir=f"{base}/yearbook_results"
)



===== 📘 Processing Senior Class =====

--- Searching for Edith  Allison (female) ---

--- Searching for ['edith'] [''] ['allison'] | Gender: female ---
Expected birth years: 1920–1926
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Iola Bobette Anders (female) ---

--- Searching for ['iola'] ['', 'bobette', 'b'] ['anders'] | Gender: female ---
Expected birth years: 1920–1926
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Julia Jeanette Anders (female) ---

--- Searching for ['julia'] ['', 'jeanette', 'j'] ['anders'] | Gender: female ---
Expected birth years: 1920–1926
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Charlie G Banks (male) ---

--- Searching for ['charlie'] ['', 'g'] ['banks'] | Gender: male ---
Expected birth years: 1920–1926
📂 Loading relevant subset from Parquet...
✅ Found 2 matching name records.

--- Searching for Albert Monroe 

,person_name,gender_guess,class_level,byear
0,Charlie G Banks,male,Senior,1922
1,Charlie G Banks,male,Senior,1924
2,Albert Monroe Cantrell,male,Senior,1921
3,George W Gibbs,male,Senior,1920
4,George W Gibbs,male,Senior,1921
5,George W Gibbs,male,Senior,1924
6,Margaret Arlola Jenkins,female,Senior,1922
7,Margaret Arlola Jenkins,female,Senior,1922
8,Margaret Arlola Jenkins,female,Senior,1923
9,Margaret Arlola Jenkins,female,Senior,1923



✅ Combined 65 total matches across all classes!

📊 Summary by class:


,class_level,total_matches
0,Freshman,4
1,Junior,4
2,Senior,10
3,Sophomore,47


In [73]:
#chief juniors old funtion
yearbook_year = (1939+1)
base = "/Users/nickichen/Desktop/urap data"

all_results = []

# === All people and corresponding genders (chief 1939 soph) ===
sophomores = [
    ("imogene", "", "allen"), ("wanda", "", "anders"), ("maudina", "", "bell"), ("hamilton", "", "boyd"),
    ("cecil", "", "cairnes"), ("irene", "", "corn"), ("eddie", "lee", "corn"), ("opal", "", "dalton"),
    ("coy", "", "dalton"), ("palmer", "", "drake"), ("ellis", "", "garren"), ("plato", "", "hall"),
    ("maxine", "", "hollingsworth"), ("hamilton", "", "hollingsworth"), ("ralph", "", "jones"),
    ("j", "b", "laughter"), ("wade", "", "laughter"), ("charles", "", "ledbetter"), ("freda", "", "marona"),
    ("joyce", "", "mckinna"), ("lottie", "", "mintz"), ("dorothy", "", "simpson"), ("ruth", "", "stamey"),
    ("glen", "", "simpson"), ("lillie", "mae", "smith")
]
sophomore_genders = [
    "female","female","female","male","male","female","female","female","male","male","male","male",
    "female","male","male","male","male","male","female","female","female","female","female","male","female"
]

# === Run the searches ===
for (first, middle, last), gender in zip(people, genders):
    print(f"\n--- Searching for {first.title()} {middle.title()} {last.title()} ({gender}) ---")

    df = find_yearbook_person_fast(
        first_names=first,
        middle_names=middle,
        last_names=last,
        gender=gender,
        yearbook_year=yearbook_year,
        cleaned_names=cleaned_names,
        geography=geography,
        database_path=f"{base}/bunmd_sorted.parquet"
    )

    if not df.empty:
        df["person_name"] = f"{first.title()} {middle.title()} {last.title()}".replace("  ", " ")
        df["gender"] = gender
        df["yearbook"] = "The Chief 1939"
        all_results.append(df)

# === Combine everything ===
if all_results:
    combined_results = pd.concat(all_results, ignore_index=True)
    print(f"\n✅ Combined {len(all_results)} matched individuals from chief sophmores 1939.")
    display(combined_results.head(100))
else:
    print("⚠️ No matches found for any listed names.")


--- Searching for O A  Anders (male) ---

--- Searching for ['o a'] [''] ['anders'] | Gender: male ---
Expected birth years: 1921–1927
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for C W  Adoeck (male) ---

--- Searching for ['c w'] [''] ['adoeck'] | Gender: male ---
Expected birth years: 1921–1927
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Franklin  Allison (male) ---

--- Searching for ['franklin'] [''] ['allison'] | Gender: male ---
Expected birth years: 1921–1927
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Broaddeth  Blackwell (male) ---

--- Searching for ['broaddeth'] [''] ['blackwell'] | Gender: male ---
Expected birth years: 1921–1927
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Claudine  Cairnes (female) ---

--- Searching for ['claudine'] [''] ['cairnes'] | Gender: female ---
Expect

In [72]:
# chief juniors

import pandas as pd

yearbook_year = 1939
base = "/Users/nickichen/Desktop/urap data"

all_results = []

# All people and corresponding genders
juniors = [
    ("o", "a", "anders"), ("c", "w", "adcock"), ("franklin", "", "allison"), ("broaddeth", "", "blackwell"),
    ("claudine", "", "cairnes"), ("nellie", "", "cairnes"), ("charlotte", "", "clayton"), ("salome", "", "clayton"),
    ("jackie", "", "corn"), ("vernon", "", "conner"), ("opal", "", "dalton"), ("christine", "", "davis"),
    ("ruth", "", "gonce"), ("eula", "", "gray"), ("willie", "", "gash"), ("blanche", "", "hollingsworth"),
    ("cameron", "", "houk"), ("helen", "", "mckinna"), ("raymond", "", "mccarson"), ("alma", "", "merrill"),
    ("carolyn", "", "orr"), ("thelma", "", "snyder"), ("jesse", "lee", "simpson"), ("james", "", "smith"),
    ("clarence", "", "williams"), ("helen", "", "wilson"), ("lillian", "", "wilson"), ("lucille", "", "wilson")
]

genders = [
     "male","male","male","male","female","female","female","female","female","male","female","female",
    "female","female","male","female","male","female","male","female","female","female","male","male",
    "male","female","female","female"
]

# === Run the searches ===
for (first, middle, last), gender in zip(people, genders):
    print(f"\n--- Searching for {first.title()} {middle.title()} {last.title()} ({gender}) ---")

    df = find_yearbook_person_fast(
        first_names=first,
        middle_names=middle,
        last_names=last,
        gender=gender,
        yearbook_year=yearbook_year,
        cleaned_names=cleaned_names,
        geography=geography,
        database_path=f"{base}/bunmd.csv"

    )
     

    if not df.empty:
        df["person_name"] = f"{first.title()} {middle.title()} {last.title()}".replace("  ", " ")
        df["gender"] = gender
        df["yearbook"] = "The Chief 1939"
        all_results.append(df)

# === Combine everything ===
if all_results:
    combined_results = pd.concat(all_results, ignore_index=True)
    print(f"\n✅ Combined {len(all_results)} matched individuals from The Chief 1939.")
    display(combined_results.head(100))
else:
    print("⚠️ No matches found for any listed names.")


--- Searching for O A  Anders (male) ---

--- Searching for ['o a'] [''] ['anders'] | Gender: male ---
Expected birth years: 1920–1926
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for C W  Adoeck (male) ---

--- Searching for ['c w'] [''] ['adoeck'] | Gender: male ---
Expected birth years: 1920–1926
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Franklin  Allison (male) ---

--- Searching for ['franklin'] [''] ['allison'] | Gender: male ---
Expected birth years: 1920–1926
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Broaddeth  Blackwell (male) ---

--- Searching for ['broaddeth'] [''] ['blackwell'] | Gender: male ---
Expected birth years: 1920–1926
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Claudine  Cairnes (female) ---

--- Searching for ['claudine'] [''] ['cairnes'] | Gender: female ---
Expect

In [70]:
#crossroads 1941

# --- Seniors ---
seniors = [
    ("nancy", "", "acuff"), ("fred", "", "albright"), ("john", "", "auten"),
    ("josephine", "", "beaver"), ("joseph", "lafayette", "blackwell"),("doris", "", "camp"), 
    ("lena", "", "chandler"), ("margaret", "", "deese"),("edna", "", "efird"),
     ("willie", "frances", "efird"), ("clegg", "milton", "furr"), ("bill", "", "gantt"), 
     ("bailey", "", "gulledge"), ("margaret", "sue", "laton"),("betty", "van", "lefler"), 
     ("deward", "", "lefler"), ("ray", "", "lefler"), ("ralph", "", "lipe"), 
     ("polly", "", "martin"), ("margaret", "", "moose"), ("william", "whitfield", "morrow"), 
     ("dorothy", "", "parker"), ("laura", "frances", "peck"), ("h", "a", "perry"), 
     ("dorothy", "creason", "rhinehardt"), ("martha", "", "russell"), ("louise", "", "shoe"), 
     ("lloyd", "j", "skidmore"), ("mary", "ann", "skidmore"),("hilda", "", "smith"), 
     ("lillian", "marie", "talbert"), ("ben", "", "thomas"), ("james", "louis", "wallace"), ("horace", "", "whitley")
]
senior_genders = [
    "female", "male", "male",
    "female", "male", "female",
    "female", "female", "female",
    "male", "female", "female",
    "male", "male", "male",
    "female", "female", "male",
    "female", "female", "female",
    "female", "female", "female",
    "male", "female", "female",
    "female", "male", "female",
    "female", "male", "male", "male"
]


# --- Juniors ---
juniors = [
    ("jewell", "", "almond"), ("newell", "clay", "almond"), ("mary", "jane", "auten"),
    ("marilyn", "theresa", "barger"), ("frances", "ramelle", "burleson"), ("ruth", "", "cox"),
    ("patty", "mccain", "crowell"), ("nell", "efird", "denning"), ("eileen", "", "dry"),
    ("george", "hampton", "efird"), ("maxine", "", "cashatt"), ("mary", "emily", "efird"),
    ("nell", "jack", "fesperman"), ("ann", "", "henning"), ("hubert", "", "hatley"),
    ("betty", "", "hatley"), ("laura", "ellen", "hearne"), ("ruth", "", "hill"),
    ("grady", "", "hinson"), ("lois", "evelyn", "hitchcock"), ("clyde", "", "josey"),
    ("ruth", "", "king"), ("rubye", "", "lefler"), ("mary", "louise", "lisk"),
    ("bill", "", "long"), ("hallie", "", "lowder"), ("ramelle", "", "lowder"),
    ("robert", "", "lowder"), ("sybil", "yvonne", "lowder"), ("virgie", "geraldine", "luther")
]
junior_genders = [
    "female", "female", "female",
    "female", "female", "female",
    "female", "female", "female",
    "male", "female", "female",
    "male", "female", "male",
    "female", "male", "female",
    "female", "female", "female",
    "male", "female", "female",
    "female", "male", "female",
    "female", "male", "female", "female"
]


# --- Sophomores ---
sophomores = [
    ("geraldine", "", "luther"), ("lois", "", "melton"), ("evalyn", "", "merritt"),
    ("beaupine", "", "miller"), ("rachel", "", "mills"), ("helen", "", "moore"),
    ("margaret", "", "morris"), ("mazel", "", "morris"), ("ramelle", "", "morris"),
    ("bob", "", "morrow"), ("alfred", "", "morton"), ("christine", "", "morton"),
    ("estelle", "", "morton"), ("hazel", "", "morton"), ("elbert", "", "mullins"),
    ("helen", "", "napier"), ("margaret", "", "nisbet"), ("virginia", "", "nuck"),
    ("lucille", "", "palmer"), ("martha", "alice", "parrish"), ("ramelle", "", "picker"),
    ("stacy", "", "sanker"), ("bessie", "lee", "rudisill"), ("helen", "", "russell"),
    ("cathryn", "", "self"), ("ruby", "", "snider"), ("robbie", "", "snipes"),
    ("alice", "", "smith"), ("katie", "bell", "smith"), ("virginia", "", "speed"),
    ("florence", "", "stude"), ("annette", "", "stude"), ("hubert", "", "stirewalt"),
    ("jewell", "", "stoker"), ("odessa", "", "talbert"), ("louise", "", "thompson")
]
sophomore_genders = [
    "female", "female", "female",
    "female", "female", "female",
    "female", "female", "female",
    "male", "male", "female",
    "female", "male", "female",
    "female", "female", "female",
    "female", "female", "female",
    "female", "female", "female",
    "male", "female", "female",
    "female", "female", "female",
    "male", "female", "female",
    "female", "female", "female"
]


# --- Freshmen ---
freshmen = [
    ("mildred", "", "austin"), ("lorene", "", "burleson"), ("estelle", "", "lucas"),
    ("vivian", "", "comer"), ("lucienne", "", "conner"), ("margaret", "", "morton"),
    ("lucille", "", "morton"), ("leon", "", "morton"), ("robert", "", "morton"),
    ("hazel", "", "norris"), ("jean", "", "norris"), ("catherine", "", "norris"),
    ("pearl", "", "norris"), ("margaret", "", "morton"), ("jack", "", "norris"),
    ("kathleen", "", "morton"), ("margaret", "", "coggins"), ("margie", "", "morton"),
    ("hilda", "", "norris"), ("mary", "", "russell"), ("lucille", "", "morton")
]
freshmen_genders = [
    "male", "female", "female",
    "female", "female", "female",
    "male", "male", "female",
    "female", "female", "female",
    "female", "male", "female",
    "female", "female", "female",
    "female", "female", "female"
]



# === RUN ===
base = "/Users/nickichen/Desktop/urap data"
yearbook_year = 1941

class_groups = [
    ("senior", seniors, genders),
    ("junior", juniors, junior_genders),
    ("sophomore", sophomores, sophomore_genders),
    ("freshman", freshmen, freshmen_genders)
]

combined_all, summary = find_yearbook_all_classes(
    yearbook_year=yearbook_year,
    class_groups=class_groups,
    database_path=f"{base}/bunmd_sorted.parquet",
    save_dir=f"{base}/yearbook_results"
)




===== 📘 Processing Senior Class =====

--- Searching for Nancy  Acuff (male) ---

--- Searching for ['nancy'] [''] ['acuff'] | Gender: male ---
Expected birth years: 1922–1928
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Fred  Albright (male) ---

--- Searching for ['fred'] [''] ['albright'] | Gender: male ---
Expected birth years: 1922–1928
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for John  Auten (male) ---

--- Searching for ['john'] [''] ['auten'] | Gender: male ---
Expected birth years: 1922–1928
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Josephine  Beaver (male) ---

--- Searching for ['josephine'] [''] ['beaver'] | Gender: male ---
Expected birth years: 1922–1928
📂 Loading relevant subset from Parquet...
✅ Found 0 matching name records.

--- Searching for Joseph Lafayette Blackwell (female) ---

--- Searching for ['joseph'] ['', 'la

,person_name,gender_guess,class_level,byear
0,Joseph Lafayette Blackwell,female,Senior,1923
1,William Whitfield Morrow,female,Senior,1923
2,William Whitfield Morrow,female,Senior,1926
3,William Whitfield Morrow,female,Senior,1928
4,Nell Efird Denning,female,Junior,1924
5,Katie Bell Smith,female,Sophomore,1923
6,Katie Bell Smith,female,Sophomore,1924



✅ Combined 7 total matches across all classes!

📊 Summary by class:


,class_level,total_matches
0,Junior,1
1,Senior,4
2,Sophomore,2


In [ ]:
# new idea: nickname matches? typos? 
MALE_NICKNAMES = {
    "william": ["will", "bill", "billy"],
    "robert": ["rob", "bob", "bobby"],
    "john": ["jack", "johnny"],
    "james": ["jim", "jimmy"],
    "charles": ["charlie"],
    "henry": ["harry"],
    "edward": ["ed", "eddie"],
    "joseph": ["joe"],
    "thomas": ["tom"],
    "richard": ["rich", "rick", "dick"],
    "frank": ["frankie"],
    "george": ["georgie"]
}


FEMALE_NICKNAMES = {
    "margaret": ["maggie", "meg", "peggy"],
    "elizabeth": ["liz", "lizzie", "beth", "betty"],
    "katherine": ["kate", "katie"],
    "catherine": ["cate", "katie"],
    "mary": ["polly", "mamie"],
    "anna": ["annie"],
    "helen": ["nell", "nellie"],
    "dorothy": ["dot", "dottie"],
    "florence": ["flossie"],
    "louise": ["lou"],
    "martha": ["patty"],
    "virginia": ["ginny"],
    "josephine": ["josie"],
    "frances": ["frankie"]
}



In [83]:
!pip install rapidfuzz


Python(57419) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: pip install --upgrade pip


In [8]:
#new function using exact, (nicknames), 

# ================================
# Yearbook → Numident Matching
# ================================
import pandas as pd
import numpy as np
import os
from rapidfuzz import fuzz

# ----------------
# Nicknames (1900–1960 safe)
# ----------------
NICKNAMES = {
    "william": ["bill", "billy", "will"],
    "robert": ["bob", "bobby"],
    "james": ["jim", "jimmy"],
    "john": ["jack"],
    "margaret": ["maggie", "peg", "peggy"],
    "elizabeth": ["liz", "beth", "betty"],
    "mary": ["polly", "molly"],
    "charles": ["charlie"],
    "thomas": ["tom"],
    "edward": ["ed", "eddie"]
}

def expand_first_names(name):
    name = name.lower().strip()
    names = {name}
    if name in NICKNAMES:
        names.update(NICKNAMES[name])
    return list(names)

# ----------------
# Single-person matcher
# ----------------
def find_yearbook_person_fast(
    first,
    last,
    gender,
    yearbook_year,
    database_path,
    fuzzy_threshold=92
):
    # Birth year window (baked in early)
    birth_low = yearbook_year - 19
    birth_high = yearbook_year - 13

    # Load only relevant rows
    db = pd.read_parquet(
        database_path,
        filters=[("byear", ">=", birth_low), ("byear", "<=", birth_high)],
        engine="pyarrow"
    )

    first = first.lower().strip()
    last = last.lower().strip()

    # Female surname flexibility
    last_names = {last}

    # -------- PASS 1: exact --------
    exact = db[
        (db["fname"].str.lower() == first) &
        (db["lname"].str.lower().isin(last_names))
    ].copy()

    if not exact.empty:
        exact["match_method"] = "exact"
        return exact

    # -------- PASS 2: nickname --------
    first_variants = expand_first_names(first)

    nick = db[
        (db["fname"].str.lower().isin(first_variants)) &
        (db["lname"].str.lower().isin(last_names))
    ].copy()

    if not nick.empty:
        nick["match_method"] = "nickname"
        return nick

    # -------- PASS 3: fuzzy (last resort) --------
    candidates = db[db["lname"].str.lower().isin(last_names)].copy()

    def score(row):
        fname = row["fname"]
        if not isinstance(fname, str):
            return 0
        fname = fname.lower()
        return max(fuzz.ratio(fname, f) for f in first_variants)

    candidates["fuzzy_score"] = candidates.apply(score, axis=1)
    fuzzy = candidates[candidates["fuzzy_score"] >= fuzzy_threshold]

    if not fuzzy.empty:
        fuzzy["match_method"] = "fuzzy"
        return fuzzy

    return pd.DataFrame()

# ----------------
# Whole-yearbook runner
# ----------------
def find_yearbook_all_classes(
    yearbook_year,
    yearbook_name,
    class_groups,
    database_path,
    save_dir
):
    all_results = []

    for class_name, people, genders in class_groups:
        print(f"\n===== Processing {class_name.title()} =====")

        for (first, _, last), gender in zip(people, genders):
            result = find_yearbook_person_fast(
                first=first,
                last=last,
                gender=gender,
                yearbook_year=yearbook_year,
                database_path=database_path
            )

            if not result.empty:
                result["yearbook_person"] = f"{first.title()} {last.title()}"
                result["class_level"] = class_name
                result["gender_guess"] = gender
                result["yearbook"] = f"{yearbook_name} {yearbook_year}"
                all_results.append(result)

    if not all_results:
        print("No matches found.")
        return None, None

    combined = pd.concat(all_results, ignore_index=True)

    # -------- Summary --------
    summary = (
        combined.groupby("yearbook_person")
        .agg(
            total_matches=("ssn", "count"),
            best_match=("match_method", "min")
        )
        .reset_index()
    )

    os.makedirs(save_dir, exist_ok=True)
    combined.to_csv(f"{save_dir}/{yearbook_name}_{yearbook_year}_matches.csv", index=False)
    summary.to_csv(f"{save_dir}/{yearbook_name}_{yearbook_year}_summary.csv", index=False)

    print(f"\nMatched {summary.shape[0]} unique yearbook individuals.")
    return combined, summary


In [9]:

 # ================================
# Crossroads High School – 1941
# ================================

yearbook_name = "Crossroads"
yearbook_year = 1941
base = "/Users/nickichen/Desktop/urap data"

# ---------- Seniors ----------
seniors = [
    ("nancy", "", "acuff"), ("fred", "", "albright"), ("john", "", "auten"),
    ("josephine", "", "beaver"), ("joseph", "lafayette", "blackwell"), ("doris", "", "camp"),
    ("lena", "", "chandler"), ("margaret", "", "deese"), ("edna", "", "efird"),
    ("willie", "frances", "efird"), ("clegg", "milton", "furr"), ("bill", "", "gantt"),
    ("bailey", "", "gulledge"), ("margaret", "sue", "laton"), ("betty", "van", "lefler"),
    ("deward", "", "lefler"), ("ray", "", "lefler"), ("ralph", "", "lipe"),
    ("polly", "", "martin"), ("margaret", "", "moose"), ("william", "whitfield", "morrow"),
    ("dorothy", "", "parker"), ("laura", "frances", "peck"), ("h", "a", "perry"),
    ("dorothy", "creason", "rhinehardt"), ("martha", "", "russell"), ("louise", "", "shoe"),
    ("lloyd", "j", "skidmore"), ("mary", "ann", "skidmore"), ("hilda", "", "smith"),
    ("lillian", "marie", "talbert"), ("ben", "", "thomas"),
    ("james", "louis", "wallace"), ("horace", "", "whitley")
]

senior_genders = [
    "female","male","male","female","male","female","female","female","female",
    "male","female","female","male","male","male","female","female","male",
    "female","female","female","female","female","female","male","female","female",
    "female","male","female","female","male","male","male"
]

# ---------- Juniors ----------
juniors = [
    ("jewell", "", "almond"), ("newell", "clay", "almond"), ("mary", "jane", "auten"),
    ("marilyn", "theresa", "barger"), ("frances", "ramelle", "burleson"), ("ruth", "", "cox"),
    ("patty", "mccain", "crowell"), ("nell", "efird", "denning"), ("eileen", "", "dry"),
    ("george", "hampton", "efird"), ("maxine", "", "cashatt"), ("mary", "emily", "efird"),
    ("nell", "jack", "fesperman"), ("ann", "", "henning"), ("hubert", "", "hatley"),
    ("betty", "", "hatley"), ("laura", "ellen", "hearne"), ("ruth", "", "hill"),
    ("grady", "", "hinson"), ("lois", "evelyn", "hitchcock"), ("clyde", "", "josey"),
    ("ruth", "", "king"), ("rubye", "", "lefler"), ("mary", "louise", "lisk"),
    ("bill", "", "long"), ("hallie", "", "lowder"), ("ramelle", "", "lowder"),
    ("robert", "", "lowder"), ("sybil", "yvonne", "lowder"), ("virgie", "geraldine", "luther")
]

junior_genders = [
    "female","female","female","female","female","female","female","female","female",
    "male","female","female","male","female","male","female","male","female",
    "female","female","female","male","female","female","female","male","female",
    "female","male","female","female"
]

# ---------- Sophomores ----------
sophomores = [
    ("geraldine", "", "luther"), ("lois", "", "melton"), ("evalyn", "", "merritt"),
    ("beaupine", "", "miller"), ("rachel", "", "mills"), ("helen", "", "moore"),
    ("margaret", "", "morris"), ("mazel", "", "morris"), ("ramelle", "", "morris"),
    ("bob", "", "morrow"), ("alfred", "", "morton"), ("christine", "", "morton"),
    ("estelle", "", "morton"), ("hazel", "", "morton"), ("elbert", "", "mullins"),
    ("helen", "", "napier"), ("margaret", "", "nisbet"), ("virginia", "", "nuck"),
    ("lucille", "", "palmer"), ("martha", "alice", "parrish"), ("ramelle", "", "picker"),
    ("stacy", "", "sanker"), ("bessie", "lee", "rudisill"), ("helen", "", "russell"),
    ("cathryn", "", "self"), ("ruby", "", "snider"), ("robbie", "", "snipes"),
    ("alice", "", "smith"), ("katie", "bell", "smith"), ("virginia", "", "speed"),
    ("florence", "", "stude"), ("annette", "", "stude"), ("hubert", "", "stirewalt"),
    ("jewell", "", "stoker"), ("odessa", "", "talbert"), ("louise", "", "thompson")
]

sophomore_genders = [
    "female","female","female","female","female","female","female","female","female",
    "male","male","female","female","male","female","female","female","female",
    "female","female","female","female","female","female","male","female","female",
    "female","female","female","male","female","female","female","female","female"
]

# ---------- Freshmen ----------
freshmen = [
    ("mildred", "", "austin"), ("lorene", "", "burleson"), ("estelle", "", "lucas"),
    ("vivian", "", "comer"), ("lucienne", "", "conner"), ("margaret", "", "morton"),
    ("lucille", "", "morton"), ("leon", "", "morton"), ("robert", "", "morton"),
    ("hazel", "", "norris"), ("jean", "", "norris"), ("catherine", "", "norris"),
    ("pearl", "", "norris"), ("margaret", "", "morton"), ("jack", "", "norris"),
    ("kathleen", "", "morton"), ("margaret", "", "coggins"), ("margie", "", "morton"),
    ("hilda", "", "norris"), ("mary", "", "russell"), ("lucille", "", "morton")
]

freshmen_genders = [
    "male","female","female","female","female","female","male","male","female",
    "female","female","female","female","male","female","female","female","female",
    "female","female","female"
]

# ---------- Group & Run ----------
class_groups = [
    ("senior", seniors, senior_genders),
    ("junior", juniors, junior_genders),
    ("sophomore", sophomores, sophomore_genders),
    ("freshman", freshmen, freshmen_genders)
]

combined_all, summary = find_yearbook_all_classes(
    yearbook_year=yearbook_year,
    yearbook_name=yearbook_name,
    class_groups=class_groups,
    database_path=f"{base}/bunmd_sorted.parquet",
    save_dir=f"{base}/yearbook_results"
)



===== Processing Senior =====

===== Processing Junior =====

===== Processing Sophomore =====

===== Processing Freshman =====

Matched 72 unique yearbook individuals.


In [2]:
import pandas as pd

path = "/Users/nickichen/Desktop/urap data/yearbook_results/Crossroads_1941_matches.csv"
df = pd.read_csv(path)

df_nc = df[df['socstate_string'] == "North Carolina"]

print(df_nc.shape)
df_nc

(121, 40)


,ssn,zip_residence,lname,mname,fname,byear,dyear,socstate,bmonth,dmonth,...,death_age,weight,ccweight,bpl_string,socstate_string,match_method,yearbook_person,class_level,gender_guess,yearbook
4,244187254,NaN,AUTEN,NaN,JOHN,1922,1978,3700.0,12.0,5,...,55,NaN,NaN,North Carolina,North Carolina,exact,John Auten,senior,male,Crossroads 1941
5,245386791,280015014,AUTEN,A,JOHN,1922,1993,3700.0,5.0,5,...,70,1.030643,1.118882,North Carolina,North Carolina,exact,John Auten,senior,male,Crossroads 1941
6,245220841,NaN,AUTEN,NaN,JOHN,1925,1981,3700.0,9.0,5,...,55,NaN,NaN,North Carolina,North Carolina,exact,John Auten,senior,male,Crossroads 1941
14,239306877,NaN,BLACKWELL,NaN,JOSEPH,1926,1974,3700.0,2.0,2,...,47,NaN,NaN,NaN,North Carolina,exact,Joseph Blackwell,senior,male,Crossroads 1941
30,244364990,280017374,LEFLER,A,BETTY,1928,2007,3700.0,11.0,9,...,78,NaN,NaN,North Carolina,North Carolina,exact,Betty Lefler,senior,male,Crossroads 1941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906,242242019,224076661,RUSSELL,P,MARY,1923,1999,3700.0,2.0,2,...,75,1.034287,1.072181,North Carolina,North Carolina,exact,Mary Russell,freshman,female,Crossroads 1941
1909,242302324,233242847,RUSSELL,M,MARY,1923,2006,3700.0,3.0,11,...,83,NaN,NaN,North Carolina,North Carolina,exact,Mary Russell,freshman,female,Crossroads 1941
1955,246269366,273611075,RUSSELL,L,MARY,1924,1995,3700.0,8.0,7,...,70,1.038555,1.083938,North Carolina,North Carolina,exact,Mary Russell,freshman,female,Crossroads 1941
2004,242349613,292041607,RUSSELL,M,MARY,1927,1993,3700.0,4.0,4,...,65,1.060677,1.100685,North Carolina,North Carolina,exact,Mary Russell,freshman,female,Crossroads 1941


In [24]:
df_counts = (
    df.groupby("yearbook_person")
      .size()
      .reset_index(name="num_matches")
      .sort_values("num_matches", ascending=False)
)

df_counts

,yearbook_person,num_matches
1,Alice Smith,270
29,James Wallace,216
23,Helen Moore,200
55,Mary Russell,171
12,Dorothy Parker,165
...,...,...
42,Louise Shoe,1
41,Lorene Burleson,1
39,Lois Hitchcock,1
38,Lloyd Skidmore,1


In [49]:
df_class = df.groupby("class_level")["yearbook_person"].nunique()
df_class


class_level
freshman     15
junior       13
senior       22
sophomore    22
Name: yearbook_person, dtype: int64

In [ ]:
#zoom into why couldnt find. filter more

#commit onto git 

In [6]:
import pandas as pd
from difflib import get_close_matches

# ---------- your yearbook tuples ----------
yearbook = [
    ("nancy", "", "acuff"), ("fred", "", "albright"), ("john", "", "auten"),
    ("josephine", "", "beaver"), ("joseph", "lafayette", "blackwell"), ("doris", "", "camp"),
    ("lena", "", "chandler"), ("margaret", "", "deese"), ("edna", "", "efird"),
    ("willie", "frances", "efird"), ("clegg", "milton", "furr"), ("bill", "", "gantt"),
    ("bailey", "", "gulledge"), ("margaret", "sue", "laton"), ("betty", "van", "lefler"),
    ("deward", "", "lefler"), ("ray", "", "lefler"), ("ralph", "", "lipe"),
    ("polly", "", "martin"), ("margaret", "", "moose"), ("william", "whitfield", "morrow"),
    ("dorothy", "", "parker"), ("laura", "frances", "peck"), ("h", "a", "perry"),
    ("dorothy", "creason", "rhinehardt"), ("martha", "", "russell"), ("louise", "", "shoe"),
    ("lloyd", "j", "skidmore"), ("mary", "ann", "skidmore"), ("hilda", "", "smith"),
    ("lillian", "marie", "talbert"), ("ben", "", "thomas"),
    ("james", "louis", "wallace"), ("horace", "", "whitley"), ("jewell", "", "almond"),
    ("newell", "clay", "almond"), ("mary", "jane", "auten"),
    ("marilyn", "theresa", "barger"), ("frances", "ramelle", "burleson"), ("ruth", "", "cox"),
    ("patty", "mccain", "crowell"), ("nell", "efird", "denning"), ("eileen", "", "dry"),
    ("george", "hampton", "efird"), ("maxine", "", "cashatt"), ("mary", "emily", "efird"),
    ("nell", "jack", "fesperman"), ("ann", "", "henning"), ("hubert", "", "hatley"),
    ("betty", "", "hatley"), ("laura", "ellen", "hearne"), ("ruth", "", "hill"),
    ("grady", "", "hinson"), ("lois", "evelyn", "hitchcock"), ("clyde", "", "josey"),
    ("ruth", "", "king"), ("rubye", "", "lefler"), ("mary", "louise", "lisk"),
    ("bill", "", "long"), ("hallie", "", "lowder"), ("ramelle", "", "lowder"),
    ("robert", "", "lowder"), ("sybil", "yvonne", "lowder"),
    ("virgie", "geraldine", "luther"), ("geraldine", "", "luther"), ("lois", "", "melton"),
    ("evalyn", "", "merritt"), ("beaupine", "", "miller"), ("rachel", "", "mills"),
    ("helen", "", "moore"), ("mildred", "", "austin"), ("lorene", "", "burleson"),
    ("estelle", "", "lucas"), ("vivian", "", "comer"), ("lucienne", "", "conner"),
    ("margaret", "", "morton"), ("lucille", "", "morton"), ("leon", "", "morton"),
    ("robert", "", "morton"), ("hazel", "", "norris"), ("jean", "", "norris"),
    ("catherine", "", "norris"), ("pearl", "", "norris"), ("margaret", "", "morton"),
    ("jack", "", "norris"), ("kathleen", "", "morton"), ("margaret", "", "coggins"),
    ("margie", "", "morton"), ("hilda", "", "norris"), ("mary", "", "russell"),
    ("lucille", "", "morton"), ("margaret", "", "morris"), ("mazel", "", "morris"),
    ("ramelle", "", "morris"), ("bob", "", "morrow"), ("alfred", "", "morton"),
    ("christine", "", "morton"), ("estelle", "", "morton"), ("hazel", "", "morton"),
    ("elbert", "", "mullins"), ("helen", "", "napier"), ("margaret", "", "nisbet"),
    ("virginia", "", "nuck"), ("lucille", "", "palmer"), ("martha", "alice", "parrish"),
    ("ramelle", "", "picker"), ("stacy", "", "sanker"), ("bessie", "lee", "rudisill"),
    ("helen", "", "russell"), ("cathryn", "", "self"), ("ruby", "", "snider"),
    ("robbie", "", "snipes"), ("alice", "", "smith"), ("katie", "bell", "smith"),
    ("virginia", "", "speed"), ("florence", "", "stude"), ("annette", "", "stude"),
    ("hubert", "", "stirewalt"), ("jewell", "", "stoker"), ("odessa", "", "talbert"),
    ("louise", "", "thompson")
]

# ---------- helper to normalize strings ----------
def normalize(s):
    return (
        str(s)
        .strip()
        .lower()
        .replace(".", "")      # remove periods in initials
        .replace(",", "")      # remove commas
        .replace("  ", " ")
    )

# ---------- 1) build yearbook df using FIRST + LAST only ----------
yb_df = pd.DataFrame(yearbook, columns=["first", "middle", "last"])
yb_df["yearbook_person"] = (yb_df["first"].fillna("").astype(str).str.strip() + " " + yb_df["last"].fillna("").astype(str).str.strip())
yb_df["yearbook_person"] = yb_df["yearbook_person"].map(normalize)

# ---------- 2) normalize df's yearbook_person too (so both sides match) ----------
# overwrite a copy so you're safe
df_work = df.copy()
df_work["yearbook_person_norm"] = df_work["yearbook_person"].astype(str).map(normalize)

# ---------- 3) recalc counts from the NORMALIZED column ----------
df_counts = (
    df_work.groupby("yearbook_person_norm")
      .size()
      .reset_index(name="num_matches")
      .sort_values("num_matches", ascending=False)
)

# ---------- 4) merge using the normalized keys ----------
yb_with_counts = yb_df.merge(df_counts, left_on="yearbook_person", right_on="yearbook_person_norm", how="left")
yb_with_counts["num_matches"] = yb_with_counts["num_matches"].fillna(0).astype(int)

# ---------- 5) results ----------
print("Overlap count (exact match on first+last):", (yb_with_counts["num_matches"]>0).sum())
display(yb_with_counts.sort_values("num_matches", ascending=False).reset_index(drop=True))

# ---------- 6) quick fuzzy-check for some missing folks (optional) ----------
missing = yb_with_counts[yb_with_counts["num_matches"] == 0].copy()
df_names_list = sorted(df_work["yearbook_person_norm"].unique().tolist())

# show up to 5 close matches for the first 20 missing names
missing = missing.head(20).copy()
missing["close_suggestions"] = missing["yearbook_person"].apply(lambda s: get_close_matches(s, df_names_list, n=5, cutoff=0.75))
display(missing)



Overlap count (exact match on first+last): 74


,first,middle,last,yearbook_person,yearbook_person_norm,num_matches
0,alice,,smith,alice smith,alice smith,270
1,james,louis,wallace,james wallace,james wallace,216
2,helen,,moore,helen moore,helen moore,200
3,mary,,russell,mary russell,mary russell,171
4,dorothy,,parker,dorothy parker,dorothy parker,165
5,ruth,,king,ruth king,ruth king,101
6,ruth,,hill,ruth hill,ruth hill,94
7,louise,,thompson,louise thompson,louise thompson,88
8,margaret,,morris,margaret morris,margaret morris,84
9,robert,,morton,robert morton,robert morton,77


,first,middle,last,yearbook_person,yearbook_person_norm,num_matches,close_suggestions
0,nancy,,acuff,nancy acuff,NaN,0,[]
3,josephine,,beaver,josephine beaver,NaN,0,[]
8,edna,,efird,edna efird,NaN,0,[]
9,willie,frances,efird,willie efird,NaN,0,[]
10,clegg,milton,furr,clegg furr,NaN,0,[]
11,bill,,gantt,bill gantt,NaN,0,[]
12,bailey,,gulledge,bailey gulledge,NaN,0,[]
13,margaret,sue,laton,margaret laton,NaN,0,[margaret morton]
16,ray,,lefler,ray lefler,NaN,0,[]
19,margaret,,moose,margaret moose,NaN,0,"[margaret morton, margaret morris, margaret de..."


In [7]:
#try to find nancy acuff manaually
import pandas as pd

base = "/Users/nickichen/Desktop/urap data"

df = pd.read_parquet(f"{base}/bunmd_sorted.parquet")

hits = df[
    (df["fname"].astype(str).str.strip().str.lower() == "nancy") &
    (df["lname"].astype(str).str.strip().str.lower() == "acuff")
]

print("num hits:", len(hits))
display(hits.head(50))


num hits: 1


,ssn,zip_residence,lname,mname,fname,byear,dyear,socstate,bmonth,dmonth,bday,dday,number_apps,race_first,race_last,race_last_cyear,race_first_cyear,race_last_cmonth,race_first_cmonth,race_change,number_claims,sex,bpl,father_fname,father_mname,father_lname,mother_fname,mother_mname,mother_lname,age_first_application,death_age,weight,ccweight,bpl_string,socstate_string
43575051,430840108,727648589,ACUFF,L,NANCY,1945,2004,500.0,7.0,6,15.0,1.0,2,1.0,1.0,1965.0,1962.0,6.0,4.0,0.0,0,2.0,500.0,WALTER,L,PARKER,GRACE,None,WARD,16.0,58,NaN,NaN,Arkansas,Arkansas


In [8]:
import pandas as pd
from difflib import SequenceMatcher

base = "/Users/nickichen/Desktop/urap data"
df = pd.read_parquet(f"{base}/bunmd_sorted.parquet")

def norm(s):
    if pd.isna(s): 
        return ""
    return (
        str(s).strip().lower()
        .replace(".", "")
        .replace(",", "")
        .replace("'", "")
        .replace("-", " ")
    )

def sim(a, b):
    return SequenceMatcher(None, a, b).ratio()

target_last = "acuff"
# nickname-ish variants for nancy (add/remove as you like)
first_targets = [
    "nancy", "nan", "nannie", "nanny", "nance", "nancyj", "anne", "ann", "anna", "annie"
]

# normalize columns once
work = df.copy()
work["fname_n"] = work["fname"].map(norm)
work["lname_n"] = work["lname"].map(norm)

# 1) narrow down by last name similarity to keep it fast
# (also catches small OCR errors like acuf, acuffe, etc.)
work["lname_score"] = work["lname_n"].apply(lambda x: sim(x, target_last))
cand = work[work["lname_score"] >= 0.75].copy()   # loosen to 0.70 if needed

print("Candidates after last-name filter:", len(cand))

# 2) score first name vs ANY nickname variant
def best_first_score(x):
    return max(sim(x, t) for t in first_targets) if x else 0.0

cand["fname_score"] = cand["fname_n"].apply(best_first_score)

# 3) combine score (weight last name more)
cand["score"] = 0.65 * cand["lname_score"] + 0.35 * cand["fname_score"]

# 4) show top hits
cols_to_show = ["byear", "fname", "lname", "lname_score", "fname_score", "score"]
cols_to_show = [c for c in cols_to_show if c in cand.columns]

top = cand.sort_values("score", ascending=False).head(100)
display(top[cols_to_show])


Candidates after last-name filter: 2645


,byear,fname,lname,lname_score,fname_score,score
33776105,1928,ANNA,ACUFF,1.000000,1.000000,1.000000
18889,1872,ANNA,ACUFF,1.000000,1.000000,1.000000
42055541,1941,ANNA,ACUFF,1.000000,1.000000,1.000000
43575051,1945,NANCY,ACUFF,1.000000,1.000000,1.000000
4673551,1904,HANNAH,ACUFF,1.000000,0.800000,0.930000
80160,1877,HANNAH,ACUFF,1.000000,0.800000,0.930000
5380979,1905,JANIE,ACUFF,1.000000,0.800000,0.930000
21072609,1918,ANNA,CUFF,0.888889,1.000000,0.927778
583068,1890,ANNE,CUFF,0.888889,1.000000,0.927778
12826608,1912,ANNA,CUFF,0.888889,1.000000,0.927778


In [9]:
import pandas as pd
from difflib import SequenceMatcher

base = "/Users/nickichen/Desktop/urap data"
df = pd.read_parquet(f"{base}/bunmd_sorted.parquet")

# ---- 1941 HS seniors -> likely birth year window ----
# Typical: 17–19 in senior year
BYEAR_MIN, BYEAR_MAX = 1922, 1924  # widen to 1921–1925 if you want

# ---- name targets ----
target_last = "acuff"
first_targets = ["nancy", "nan", "nannie", "nanny", "nance", "ann", "anne", "annie", "anna", "nancie"]

def norm(s):
    if pd.isna(s):
        return ""
    return (
        str(s).strip().lower()
        .replace(".", "")
        .replace(",", "")
        .replace("'", "")
        .replace("-", " ")
    )

def sim(a, b):
    return SequenceMatcher(None, a, b).ratio()

# ---- normalize + filter by birth year first ----
work = df.copy()
work["fname_n"] = work["fname"].map(norm)
work["lname_n"] = work["lname"].map(norm)

work = work[
    work["byear"].between(BYEAR_MIN, BYEAR_MAX, inclusive="both")
].copy()

print("Rows in birth-year window:", len(work))

# ---- last name similarity filter ----
work["lname_score"] = work["lname_n"].apply(lambda x: sim(x, target_last))
cand = work[work["lname_score"] >= 0.70].copy()   # tighten to 0.75 if too many

# ---- first name best score across nickname variants ----
def best_first_score(x):
    return max(sim(x, t) for t in first_targets) if x else 0.0

cand["fname_score"] = cand["fname_n"].apply(best_first_score)

# ---- overall score (weight last name more) ----
cand["score"] = 0.65 * cand["lname_score"] + 0.35 * cand["fname_score"]

# ---- pick columns to display: byear + names + soc states + whatever else exists ----
want_cols = [
    "byear", "fname", "lname",
    "socstate", "socstate_string", "bplstate", "bplstate_string",
    "resstate", "resstate_string",
    "race", "sex",
    "score", "lname_score", "fname_score"
]
cols = [c for c in want_cols if c in cand.columns]

top = cand.sort_values(["score", "lname_score", "fname_score"], ascending=False).head(100)

print("Top candidates:", len(top))
display(top[cols])


Rows in birth-year window: 3927118
Top candidates: 100


,byear,fname,lname,socstate,socstate_string,sex,score,lname_score,fname_score
27821235,1923,ALINE,ACUFF,4700.0,Tennessee,2.0,0.883333,1.000000,0.666667
27821239,1923,FRANCES,ACUFF,2400.0,Maryland,2.0,0.883333,1.000000,0.666667
26472694,1922,JONATHAN,ACUFF,4800.0,Texas,1.0,0.840909,1.000000,0.545455
27821241,1923,HORACE,ACUFF,600.0,California,1.0,0.840909,1.000000,0.545455
27821245,1923,MAURINE,ACUFF,4700.0,Tennessee,2.0,0.840909,1.000000,0.545455
26472690,1922,ELENA,ACUFF,1700.0,Illinois,2.0,0.825000,1.000000,0.500000
29122746,1924,CONDA,ACUFF,4700.0,Tennessee,1.0,0.825000,1.000000,0.500000
27821244,1923,LUCILLE,ACUFF,4700.0,Tennessee,2.0,0.811538,1.000000,0.461538
26730362,1922,FRANCES,CUFF,4500.0,South Carolina,2.0,0.811111,0.888889,0.666667
26472691,1922,IRENE,ACUFF,4900.0,Utah,2.0,0.805556,1.000000,0.444444
